In [ ]:
import sys
from PyQt5 import QtWidgets, uic, QtGui, QtCore
from PyQt5.QtWidgets import QTableWidget, QTableWidgetItem, QHeaderView, QLabel, QMessageBox, QDialog, QVBoxLayout, QHBoxLayout, QLineEdit, QTextEdit, QDialogButtonBox
from PyQt5.QtWidgets import QPushButton, QGraphicsDropShadowEffect, QProgressDialog
from PyQt5.QtGui import QColor, QBrush, QPainter, QPixmap
from PyQt5.QtCore import Qt, QTimer, QRect, QThread, pyqtSignal, QSettings
import pandas as pd
import cx_Oracle
from datetime import datetime, timedelta
import subprocess
import ftplib
import os
import time
import tempfile
import shutil

In [ ]:
# Main application class
class MyWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        
    def showCampusSelectionDialog(self):
        """캠퍼스 선택 다이얼로그를 표시하고 선택된 캠퍼스를 반환"""
        dialog = QDialog(self)
        dialog.setWindowTitle("캠퍼스 선택")
        dialog.setModal(True)
        dialog.setFixedSize(400, 200)
        
        # 다이얼로그를 화면 중앙에 배치
        screen = QtWidgets.QApplication.desktop().screenGeometry()
        dialog.move((screen.width() - dialog.width()) // 2, (screen.height() - dialog.height()) // 2)
        
        layout = QVBoxLayout()
        
        # 스타일 설정
        dialog.setStyleSheet("""
            QDialog {
                background-color: #2b2b2b;
                border: 2px solid #555;
                border-radius: 10px;
                color: white;
                font-family: 'Segoe UI', Arial, sans-serif;
                font-size: 12pt;
            }
            QLabel {
                color: white;
                font-size: 14pt;
                font-weight: bold;
                margin: 10px;
            }
            QPushButton {
                background-color: #4CAF50;
                color: white;
                border: none;
                padding: 15px 20px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 14pt;
                font-weight: bold;
                margin: 5px;
                border-radius: 8px;
                min-width: 150px;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
            QPushButton:pressed {
                background-color: #3e8e41;
            }
        """)
        
        title_label = QLabel("캠퍼스를 선택해주세요")
        title_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(title_label)
        
        button_layout = QHBoxLayout()
        
        # NS2 버튼 (2캠퍼스)
        ns2_button = QPushButton("2캠퍼스 (NS2)")
        ns2_button.setStyleSheet("""
            QPushButton {
                background-color: #4169E1;
                color: white;
                border: none;
                padding: 15px 20px;
                text-align: center;
                font-size: 14pt;
                font-weight: bold;
                margin: 5px;
                border-radius: 8px;
                min-width: 150px;
            }
            QPushButton:hover {
                background-color: #365EC7;
            }
            QPushButton:pressed {
                background-color: #2E4FA7;
            }
        """)
        
        # NS3 버튼 (3캠퍼스)
        ns3_button = QPushButton("3캠퍼스 (NS3)")
        ns3_button.setStyleSheet("""
            QPushButton {
                background-color: #32CD32;
                color: white;
                border: none;
                padding: 15px 20px;
                text-align: center;
                font-size: 14pt;
                font-weight: bold;
                margin: 5px;
                border-radius: 8px;
                min-width: 150px;
            }
            QPushButton:hover {
                background-color: #2AB82A;
            }
            QPushButton:pressed {
                background-color: #22A322;
            }
        """)
        
        # 취소 버튼
        cancel_button = QPushButton("취소")
        cancel_button.setStyleSheet("""
            QPushButton {
                background-color: #DC143C;
                color: white;
                border: none;
                padding: 15px 20px;
                text-align: center;
                font-size: 14pt;
                font-weight: bold;
                margin: 5px;
                border-radius: 8px;
                min-width: 150px;
            }
            QPushButton:hover {
                background-color: #B71C1C;
            }
            QPushButton:pressed {
                background-color: #8B0000;
            }
        """)
        
        button_layout.addWidget(ns2_button)
        button_layout.addWidget(ns3_button)
        
        layout.addLayout(button_layout)
        layout.addWidget(cancel_button)
        
        dialog.setLayout(layout)
        
        selected_campus = None
        
        def select_ns2():
            nonlocal selected_campus
            selected_campus = 'NS2'
            dialog.accept()
        
        def select_ns3():
            nonlocal selected_campus
            selected_campus = 'NS3'
            dialog.accept()
        
        def cancel():
            nonlocal selected_campus
            selected_campus = None
            dialog.reject()
        
        ns2_button.clicked.connect(select_ns2)
        ns3_button.clicked.connect(select_ns3)
        cancel_button.clicked.connect(cancel)
        
        if dialog.exec_() == QDialog.Accepted:
            return selected_campus
        else:
            return None

    def showTable(self, action_type):
        """5개 버튼 클릭 시 캠퍼스 선택 후 해당 데이터를 표시하는 함수"""
        self.current_action = action_type  # 현재 액션 저장
        self.STATUS_ID.setText(action_type)  # Monitor 탭의 STATUS_ID에 텍스트 설정
        
        # 캠퍼스 선택 다이얼로그 표시
        selected_campus = self.showCampusSelectionDialog()
        
        # 사용자가 취소했으면 함수 종료
        if selected_campus is None:
            return
        
        # 캠퍼스가 선택되었으면 로딩 오버레이 표시하고 데이터 조회
        self.loading_overlay.setGeometry(self.rect())
        self.loading_overlay.show()
        
        # 선택된 캠퍼스로 executeQuery 호출
        QTimer.singleShot(100, lambda: self.executeQuery(selected_campus))
        self.closeTableButton.show()